# 第8章 了解深度學習辨識影像的方法

本章要學習的是在學習深度學習之際，執行相關程式的流程。  
若是在Google Colaboratory的環境下執行程式，請從「執行階段」點選「變更執行階段類型」，確認是否已將「硬體加速器」設定為「GPU」

※若出現錯誤訊息，可試著執行Clear Output。

In [ ]:
#Colaboratory環境の設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Chapter8

In [ ]:
#設定函式庫
!pip install -q -r ./requirements.txt

## 8-6 了解圖片構造這個學習資料


In [ ]:
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np

# 使用的是cifar10資料集。
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print("x_train.shape: ",x_train.shape)
print("y_train.shape: ",y_train.shape)
print("x_test.shape: ",x_test.shape)
print("y_test.shape: ",y_test.shape)

In [ ]:
print("shape: ",x_train[0].shape)
print(x_train[0])

In [ ]:
#顯示學習專用資料的第一張圖片
plt.imshow(x_train[0])
plt.show()

In [ ]:
#顯示學習專用資料的第一張圖片的標籤
print(y_train[0])

#列出學習專用資料、驗證資料的標籤值
print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
#讓標籤的編號與名稱配對。以標籤6為例，該值為label_names[6]，與之配對的是frog。
label_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

plt.figure(figsize=(10,5))
for index in range(10):
    img = x_train[index]
    label = label_names[y_train[index][0]]
    plt.subplot(2,5,index+1)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img)

## 8-7 利用深度學習函式庫從零開始學習圖片檔


In [ ]:
from tensorflow.keras.utils import to_categorical

#讓圖片的每個像素的值介於0～1之間
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

#執行標籤的Onehot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D

#建立模型
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu', name="conv2d_1"))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu', name="conv2d_2"))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(len(label_names), activation='softmax'))

#顯示模型的概要
print(model.summary())

model.compile(optimizer = optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 64
epochs=20

#開始學習
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=1)

In [ ]:
#計算正確率
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred,axis = 1) 
test_loss, test_acc = model.evaluate(x_test, y_test)

print(test_acc)

## 8-8 評估學習結果

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
ax[0].plot(history.history['val_loss'], color='g', label="Validation Loss")
legend = ax[0].legend()

ax[1].plot(history.history['accuracy'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_accuracy'], color='g', label="Validation Accuracy")
legend = ax[1].legend()

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns

y_pred = model.predict(x_test)
#y_pred儲存了成為各類別的機率，所以只取各類別的最大值
y_pred_classes = np.argmax(y_pred,axis = 1) 
y_true = np.argmax(y_test,axis = 1)
cf_matrix = confusion_matrix(y_true, y_pred_classes)

In [ ]:
plt.figure(figsize=(13, 13))
c = sns.heatmap(cf_matrix, annot=True,fmt="d")
c.set(xticklabels=label_names, yticklabels=label_names)
plt.plot()

## 8-9 可視化神經網路看見的「特徵」

In [ ]:
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Model
from matplotlib import colors
from PIL import Image

def grad_cam_image(model, layer_name, image):
  
  with tf.GradientTape() as tape:
    layer = model.get_layer(layer_name)

    #將輸出結果分成一般的輸出結果（輸出10個分類）與利用layer_name指定的層的輸出結果。
    tmpModel = Model([model.inputs], [model.output, layer.output])
    #model_out為輸入的圖片的分類結果。
    #layer_out為利用layer_name指定的層的輸出結果
    model_out, layer_out = tmpModel(np.array([image]))

    #將模型分類結果之中，機率最高的分類結果存入class_out
    class_out = model_out[:, np.argmax(model_out[0])]
    #計算輸出結果到指定層的斜率
    grads = tape.gradient(class_out, layer_out)
    #計算斜率的平均值。與Global Average Pooling相同的處理。
    pooled_grads = K.mean(grads, axis=(0, 1, 2))


  #讓斜率平均值與指定層的輸出結果相乘
  heatmap = tf.multiply(pooled_grads, layer_out)
  #與每個色版相加
  heatmap = tf.reduce_sum(heatmap, axis=-1)
  #避免算出負值。與ReLu相同的處理
  heatmap = np.maximum(heatmap, 0)
  #讓值限縮在0~1的範圍之內
  heatmap = heatmap/heatmap.max()

  #整理成容易瀏覽的圖片
  return_image = np.asarray(Image.fromarray(heatmap[0]).resize(image.shape[:2])) * 255
  colormap = plt.get_cmap('jet')
  return_image = return_image.reshape(-1)
  return_image = np.array([colormap(int(np.round(pixel)))[:3] for pixel in return_image]).reshape(image.shape)
  return_image = image * 0.5 + return_image * 0.5

  return return_image

In [ ]:
[layer.name for layer in model.layers]

In [ ]:
grad_cam = grad_cam_image(model, "conv2d_2", x_train[0])

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(grad_cam)

plt.subplot(1,2,2)
plt.imshow(x_train[0])
plt.show()

## 8-10 可視化完成學習後的神經網路構造

In [ ]:
def show_filters(model, layer_name):
    target_layer = model.get_layer(layer_name).get_weights()[0]
    filter_num = target_layer.shape[3]

    plt.figure(figsize=(15, 10))
    for i in range(filter_num):
        plt.subplot(int(filter_num/6) + 1, 6, i+1)
        plt.title('filter %d' % i)
        plt.axis('off')
        plt.imshow(target_layer[ :, :, 0, i], cmap="gray") 
    plt.show()
    
show_filters(model, "conv2d_1")

In [ ]:
from tensorflow.keras.models import Model

#接收模型與圖片之後，將各卷積層的輸出結果轉換成圖片再顯示
def layer_outputs(model, image):
    #只篩選卷積層
    _model = Model(inputs=model.inputs, outputs=[layer.output for layer in model.layers if type(layer) is Conv2D])

    #分類接收到的圖片
    conv_outputs = _model.predict(np.array([image]))
    
    def show_images(output, title):
        output = output[0]
        filter_num = output.shape[2]
        
        fig = plt.figure(figsize=(20, 15))
        fig.suptitle(title, size=15)
        for i in range(filter_num):
            plt.subplot(int(filter_num/8) + 1, 8, i+1)
            plt.title('filter %d' % i)
            plt.axis('off')
            plt.imshow(output[:,:,i])
    
    #輸出每個卷積層的圖片
    for i, output in enumerate(conv_outputs):
        title = "Conv layer number %d" % (i + 1)
        show_images(output, title)
        
layer_outputs(model, x_train[0])